## TensorFlow로 MNIST 훈련하기

#### 이 내용은 [tensorflow tutorial](http://www.tensorflow.org/tutorials/mnist/beginners/index.html)을 바탕으로 작성되었습니다.

input_data 모듈을 먼저 받는다. (데이터를 다루기 위해 미리 만들어놓은 python 모듈)

    $ wget https://github.com/didw/DL-TF/blob/master/input_data.py

In [4]:
# read mnist data
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


mnist는 60,000개의 mnist.training data와 10,000개의 mnist.test data로 구성되어있다.
mnist.train은 image(실제 데이터)와 labels로 구성되어있다.

mnist.train.image: 60000 x 784
mnist.train.labels: 60000 x 10



0~9의 숫자의 확률값을 추출하기 위해 softmax를 사용한다.


![softmax-regression](png/softmax-regression-scalargraph.png =500x)



In [6]:
import tensorflow as tf
x = tf.placeholder("float", [None, 784])

x는 어떤 값이 아닌 placeholder이다. (tensorflow가 시작할때 입력값을 넣기 위한 공간을 미리 할당해놓는다 정도로 이해하자.) 값이 None인 것은 임의의 입력 길이를 받기 위함이다.



W와 b를 정의 & 초기화 한다. Variable은 훈련중에 업데이트되는 tensor이다. 모델 파라메터라고 봐도 되겠다.


In [7]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

파라메터를 이용해 모델을 구현한다.


![matrix form](png/softmax-regression-vectorequation.png =500x)

In [8]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

### Training
모델을 정의했으니 이제 돌리면 된다.

아.. 그 전에 cosftFunction을 정의해야지.
여기선 cross entropy를 사용한다.
$H_{y'}(y) = -\sum_i y'_i \log(y_i)$
cross entropy는 y와 y'가 둘다 0이거나 둘다 1이면 값이 작아진다. y'는 레이블이라 당연히 0 또는 1이니, y를 y'과 같은 값을 갖도록 훈련한다고 보면 될 것 같다.
머신 러닝을 배울 때 square error를 costFunction으로 계산했던걸 보면 뭔가 더 세련되보인다.

GradientDescent 방법으로 정의해놓은 costFunction을 줄이도록 훈련방법을 정의한다.

In [12]:
y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

이제 변수를 초기화하고 세션을 만든 후 돌리면 된다.
변수초기화를 다시 하는 이유가 아마도 이전에 0으로 초기화했던걸 무시하고 DBN으로 초기화를 다시 하는게 아닌가 싶다. learning rate: 0.01

In [14]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

훈련을 할 때 전체 데이터로 훈련하지 않고 100개(1/10)를 랜덤하게 골라서 훈련을 한다. 성능차이는 별로 없고 속도는 당연히 훨씬 빨라진다. 데이터가 많아지니 온갖 꼼수를 다 쓴다.

### Evaluating
훈련 했으니 테스트를 해봐야지.

In [19]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels})

0.9132


그냥 

    print accuracy
하면 안된다.
이 문법은 익숙해지는데 시간이 좀 걸릴 것 같다.

성능이 맘에 안든다면 CNN으로..